# Temperature Ramp (5.8 -> 6.0)

In [1]:
#-----Import esential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Declare some important variables

t_0 = 5.8;
t_1 = 6.0;
p_max  = 1.4;
steps_ramp = 5e4;


#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.8_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl);
lj.set_params(mode = 'shift');

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0);

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

ramp1 = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);
npt = hoomd.md.integrate.npt(group = mesogens, kT = ramp1, tau = 5.0, tauP = 3.0, P = p_max);
npt.randomize_velocities(seed = 42);

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#-----Ramp1 T : 5.6 -> 5.8
#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [3]:
hoomd.run(steps_ramp)
system.box.get_volume()

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1450165 / 1500000 | TPS 16.4917 | ETA 00:50:21
Time 00:00:20 | Step 1450300 / 1500000 | TPS 13.4828 | ETA 01:01:26
Time 00:00:30 | Step 1450504 / 1500000 | TPS 20.3614 | ETA 00:40:30
Time 00:00:40 | Step 1450713 / 1500000 | TPS 20.7187 | ETA 00:39:38
Time 00:00:50 | Step 1450912 / 1500000 | TPS 19.8675 | ETA 00:41:10
Time 00:01:00 | Step 1451113 / 1500000 | TPS 20.0693 | ETA 00:40:35
Time 00:01:10 | Step 1451325 / 1500000 | TPS 21.0604 | ETA 00:38:31
Time 00:01:20 | Step 1451528 / 1500000 | TPS 20.2833 | ETA 00:39:49
Time 00:01:30 | Step 1451737 / 1500000 | TPS 20.892 | ETA 00:38:30
Time 00:01:40 | Step 1451940 / 1500000 | TPS 20.2928 | ETA 00:39:28
Time 00:01:50 | Step 1452145 / 1500000 | TPS 20.48 | ETA 00:38:56
Time 00:02:0

15852.41069937131

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x10a73bf28>),
             ('particles', <hoomd.data.particle_data at 0x111eb7a20>),
             ('number_density', 0.5677369941189405),
             ('bonds', <hoomd.data.bond_data at 0x111eb7a90>),
             ('angles', <hoomd.data.angle_data at 0x111eb7b00>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x111eb7b70>),
             ('impropers', <hoomd.data.dihedral_data at 0x111eb7be0>),
             ('constraints', <hoomd.data.constraint_data at 0x111eb7c50>),
             ('pairs', <hoomd.data.bond_data at 0x111eb7cc0>),
             ('timestep', 1500000)])